In [4]:
from bart_survival import simulation as sim
from bart_survival import surv_bart as sb

In [5]:
import importlib
import numpy as np
rng = np.random.default_rng(1)

In [70]:
x_mat = sim.get_x_matrix(
    N=100,
    x_vars=1,
    VAR_CLASS=[2],
    VAR_PROB=[.5],
    rng = rng
)
# x_mat

In [75]:
# x_mat = np.array([1,2]).reshape(-1,1)
event_dict, sv_true, sv_scale_true = sim.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(2 + .4*x_mat[:,0])",
    shape_f = "1",
    eos = 20,
    cens_scale=None,
    time_scale=5,
    true_only=False,
    rng = rng
)

# HR check
print(np.exp(-.4))
print(sv_true["hz_true"][0,0]/sv_true["hz_true"][1,0])

mean shape 1.0
mean scale 8.951727820066358
mean time draws 8.183425597928592
0.6703200460356392
1.0


In [86]:
# test time transform works 
t_scale = sb.get_time_transform(event_dict["t_event"], time_scale = 5)
np.alltrue(t_scale == event_dict["t_event_scale"])

True

In [111]:
importlib.reload(sb)
t_scale = sb.get_time_transform(event_dict["t_event"], time_scale = 5)
y_sk = sb.get_y_sklearn(event_dict["status"], t_scale)
trn = sb.surv_pre_train(y_sk, x_mat, weight=None)
post_test = sb.get_posterior_test(y_sk=y_sk, x_test = x_mat)


In [113]:
# # intitialize models
SPLIT_RULES =  [
    "pmb.ContinuousSplitRule()", # time
    "pmb.OneHotSplitRule", # ccsr_ind_p2
]
model_dict = {"trees": 50,
    "split_rules": SPLIT_RULES
}
sampler_dict = {
            "draws": 1000,
            "tune": 1000,
            "cores": 8,
            "chains": 8,
            "compute_convergence_checks": False
        }

In [118]:
np.unique(trn["x"][:,0])

array([1., 2., 3., 4.])

In [121]:
# event_dict["t_event_scale"]
importlib.reload(sb)
BSM = sb.BartSurvModel(model_config=model_dict, sampler_config=sampler_dict)

BSM.fit(
    y =  trn["y"],
    X = trn["x"],
    weights=trn["w"],
    coords = trn["coord"],
)


    
    


Multiprocess sampling (8 chains in 8 jobs)
PGBART: [f]


Sampling 8 chains for 1_000 tune and 1_000 draw iterations (8_000 + 8_000 draws total) took 102 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data
	> predictor_names
	> offset